<a href="https://colab.research.google.com/github/liliarql/Text-mining-exercises/blob/main/Ejercicio2_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# En caso de problemas, utilizar las dependencias de librerías de este requierement: https://github.com/googlecolab/backend-info/blob/d6d345cb94fc5fd49951c9af0f6ead5e962bfab2/pip-freeze.txt
!pip install numpy==1.23.5
!pip install transformers[torch]==4.35.2
!pip install accelerate -U
!pip install evaluate

  Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 3.4 MB/s eta 0:00:00
  Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvid

In [ ]:
import pandas as pd
def load_prepare_data(path):
  """
  Función para cargar y procesar datos para el ejercicio.
  """
  df = pd.read_csv(path,sep=",")
  map_classes = {
    "religion":1,
    "age":1,
    "ethnicity":1,
    "gender":1,
    "other_cyberbullying":1,
    "not_cyberbullying":0,
  }
  df["cyberbullying"] = df.cyberbullying_type.map(map_classes)
  return df[["tweet_text","cyberbullying"]].copy()

# Ejercicio


En este ejercicio vamos a trabajar con un conjunto de datos procedente de medios sociales online.

Uno de los mayores problemas en el internet de hoy en día es la presencia de actitudes negativas hacia algunos colectivos en relación a su etnia, género, religión o ideología política. En este ejercicio trabajaremos con un conjunto de datos reales, etiquetados manualmente, procedentes de la plataforma [Kaggle](https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification/data). Originalmente, a cada documento del dataset se le asignó una de las siguientes categorías:
- *religion*
- *age*
- *ethnicity*
- *gender*
- *other_cyberbullying*
- *not_cyberbullying*


El objetivo inicial del dataset era su uso para entrenar un modelo capaz de detectar el tipo de contenido de odio presente en internet según el colectivo al que se atacaba. En este caso, para simplificar el ejercicio, se ha generado una función `load_prepare_data()` que cambia las categorías del dataset obteníendose al final 2 categorías con valor 1 o 0, indicando si el tweet tiene contenido de odio

**En este ejercicio debeis entrenar un modelo de clasificación utilizando la librería Transformers.** Dado que el análisis exploratorio ha sido realizado en el ejercicio anterior, en este caso podréis centraros en entrenar el modelo utilizando la librería Transformers, seleccionando un modelo pre-entrenado adecuado, entrenando el modelo y llevando a cabo la evaluación.


**Nota 1**: Este ejercicio requiere el uso de las GPUs de Google Colab. Este Colab debería estar preconfigurado para ejecutarse en GPU, pero si tuviera problemas en la ejecución que me contacte a través del Moodle para buscar soluciones alternativas.

## 0. Imports


In [ ]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSequenceClassification,
   AdamW
)
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## 1. Obtención del corpus
Para la obtención de los datos teneis disponible la función `load_prepare_data()`. Esta función prepara los datos del ejercicio en formato Pandas dataframe para que podais realizarlo.

In [ ]:
path_data = "https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/cyberbullying_tweets.csv"
# Path de datos alternativos en caso de que el anterior no funcione (al estar alojado en github puede haber limitaciones
# en la descarga.
# path_data = "https://zenodo.org/records/10938455/files/cyberbullying_tweets.csv?download=1"
dataset = load_prepare_data(path_data)

In [ ]:
dataset.head(4)

,tweet_text,cyberbullying
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0


## 2. Análisis exploratorio

Podéis saltarlo en este ejercicio.

## 3. Preparación de los datos, eleccion del modelo y normalizacion

Empezamos por importar el objeto `pipeline` de la libreria `transformers`

In [ ]:
from transformers import pipeline

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Ahora creamos las variables separadas del dataset, correspondiendo respectivamente al texto del tweet (`texts`) y el indicador de si es cyberbullying o no (`cyberb`)



In [ ]:
texts = dataset.tweet_text.values
cyberb = dataset.cyberbullying.values

Realizamos la separación de nuestro dataset en muestras para entrenamiento, validación y Test (para la predicción).

In [ ]:
train_texts, test_texts, train_cyberb, test_cyberb = train_test_split(texts, cyberb, test_size=.15, random_state=0,
                                                    stratify = cyberb)
train_texts, val_texts, train_cyberb, val_cyberb = train_test_split(train_texts, train_cyberb, test_size=.15, random_state=0,stratify = train_cyberb)

Dado que el problema que se presenta a nosotros es un problema de clasificacion, vamos a probar dos modelos proveniente del hub de modelos presentes en la plataforma de HuggingFace : el modelo **bert-base-uncased** porque lo hemos visto en clase, y el modelo **twitter-roberta-base-sentiment-latest** de cardiffnlp. Cada etapa que vamos a realizar a continuacion, lo realizaremos para ambos modelos, agregando "_twrb" al segundo para poder identificarlo.

In [ ]:
model_name_1 = 'bert-base-uncased'

In [ ]:
model_name_2 = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

Cargamos los tokenizers de cada uno de los dos modelos gracias a la comanda `AutoTokenizer`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_1)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenizer_2 = AutoTokenizer.from_pretrained(model_name_2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
config = AutoConfig.from_pretrained(model_name_2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Realizamos un prueba de como funciona el tokenizador con una frase aleatoria

In [ ]:
texto = "This is a simple sentence to try the tokenizer from the model we have chosen on the HuggingFace platform."
texto_tokens = tokenizer(texto).tokens()
texto_tokens

['[CLS]',
 'this',
 'is',
 'a',
 'simple',
 'sentence',
 'to',
 'try',
 'the',
 'token',
 '##izer',
 'from',
 'the',
 'model',
 'we',
 'have',
 'chosen',
 'on',
 'the',
 'hugging',
 '##face',
 'platform',
 '.',
 '[SEP]']

Ahora generamos la funcion a continuacion. Esta funcion la usamos en un collab visto en clase y nos permite adaptar nuestros datos de nuestro dataset al formato requerido por el ecosistema de HuggingFace y asi no tener problemas a la hora de utilizar todas las funciones que derivan de los modelos que hemos elegido y extraidos de esta plataforma. La tokenizacion tambien se esta realizando a traves de ella.



In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Constructor de la clase CustomDataset.
        Parámetros:
        - texts: Lista de textos.
        - labels: Lista de etiquetas correspondientes a los textos.
        - tokenizer: Objeto del tokenizador a utilizar.
        - max_length: Longitud máxima de la secuencia después de la tokenización.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Devuelve la longitud del conjunto de datos.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Obtiene un elemento del conjunto de datos.

        Parámetros:
        - idx: Índice del elemento a obtener.

        Devuelve:
        Un diccionario con 'input_ids', 'attention_mask' y 'labels'.
        """
        # Obtener el texto y la etiqueta del índice proporcionado
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        # Tokenizar el texto
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Devolver el diccionario con los datos
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


La funcion esta cargada, ahora procedemos a transformar nuestros datos al formato Datasets:

In [ ]:
max_length = 280  # Tamano maximo de un tweet

train_dataset = CustomDataset(train_texts, train_cyberb, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_cyberb, tokenizer, max_length)
test_dataset = CustomDataset(test_texts, test_cyberb, tokenizer, max_length)

train_dataset_2 = CustomDataset(train_texts, train_cyberb, tokenizer_2, max_length)
val_dataset_2 = CustomDataset(val_texts, val_cyberb, tokenizer_2, max_length)
test_dataset_2 = CustomDataset(test_texts, test_cyberb, tokenizer_2, max_length)

In [ ]:
train_dataset[20]

{'input_ids': tensor([  101, 10166,  1037,  5637,  9040,  8257,  2061,  6616,  2378,  6057,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
train_dataset_2[20]

{'input_ids': tensor([    0, 23692,    10,  5100,  5345,  8018,    98, 46922,  6269,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

Ahora podemos pasar al entrenamiento de modelos

## 4. Entrenamiento y evaluación de modelos


Vamos a hacer uso de la clase `AutoModelForSequenceClassification` de la libreria Transformers para realizar la parte de modelizacion y entrenamiento.

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "NO_CYBERBULLYING", 1: "CYBERBULLYING"}
label2id = {"NO_CYBERBULLYING": 0, "CYBERBULLYING": 1}
model_bbu = AutoModelForSequenceClassification.from_pretrained(model_name_1,  num_labels=2, id2label=id2label, label2id=label2id)
model_twrb = AutoModelForSequenceClassification.from_pretrained(model_name_2,  num_labels=2, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification 

In [ ]:
import accelerate

training_args = TrainingArguments(
    output_dir="modelo_test",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

La siguiente **funcion de metricas** nos permitira realizar una comparativa entre los dos modelos (y tambien con el modelo final elegido del ejercicio 1) y ver cual de los 3 sale mejor.

In [ ]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)
    f1_score_value = f1_score.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_value,
        "f1_score": f1_score_value,
    }

Ajuste del modelo con el objeto `Trainer`

In [ ]:
trainer = Trainer(
    model=model_bbu,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.261700,0.260864,{'accuracy': 0.8977141917447788},{'f1': 0.9405126243305278}


Trainer is attempting to log a value of "{'accuracy': 0.8977141917447788}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9405126243305278}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=4308, training_loss=0.2747056477581226, metrics={'train_runtime': 1890.7017, 'train_samples_per_second': 18.224, 'train_steps_per_second': 2.279, 'total_flos': 4957978393888800.0, 'train_loss': 0.2747056477581226, 'epoch': 1.0})

Ajuste para el segundo modelo

In [ ]:
trainer_twrb = Trainer(
    model=model_twrb,
    args=training_args,
    train_dataset=train_dataset_2,
    eval_dataset=val_dataset_2,
    tokenizer=tokenizer_2,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
trainer_twrb.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.261300,0.275373,{'accuracy': 0.9011675711231706},{'f1': 0.942493541287915}


Trainer is attempting to log a value of "{'accuracy': 0.9011675711231706}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.942493541287915}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=4308, training_loss=0.28015478695556867, metrics={'train_runtime': 1902.5196, 'train_samples_per_second': 18.111, 'train_steps_per_second': 2.264, 'total_flos': 4957978393888800.0, 'train_loss': 0.28015478695556867, 'epoch': 1.0})

Ahora que esta entrenado el modelo, pasamos en la parte de evaluacion, usando el metodo 'evaluate' y asi sacaremos las metricas comparativas

In [ ]:
trainer.evaluate(test_dataset)

Trainer is attempting to log a value of "{'accuracy': 0.8922281241263629}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9374847968864023}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.2738875150680542,
 'eval_accuracy': {'accuracy': 0.8922281241263629},
 'eval_f1_score': {'f1': 0.9374847968864023},
 'eval_runtime': 133.567,
 'eval_samples_per_second': 53.561,
 'eval_steps_per_second': 6.701,
 'epoch': 1.0}

In [ ]:
trainer_twrb.evaluate(test_dataset_2)

Trainer is attempting to log a value of "{'accuracy': 0.8962818003913894}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9397922752353133}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.285381555557251,
 'eval_accuracy': {'accuracy': 0.8962818003913894},
 'eval_f1_score': {'f1': 0.9397922752353133},
 'eval_runtime': 124.2461,
 'eval_samples_per_second': 57.579,
 'eval_steps_per_second': 7.203,
 'epoch': 1.0}

Antes de visualizar todas las metricas de una manera mas comoda gracias a la matriz de confusion y el classification report, podemos comentar lo visto en la ejecucion anterior. El modelo 1 (Bert Base Uncased) le sale una accuracy de 89,22% y un f1-score del 93,74%. El segundo modelo sale un poquito mejor con una accuracy del 89,62% y un f1-score del 93,97%.

Por lo cual en un principio, tendemos a suponer que el modelo 2 es el que mejor sale.

Ahora usamos el `predict`del `Trainer` para obtener la prediccion sobre el dataset de test.

In [ ]:
predictions = trainer.predict(test_dataset)


In [ ]:
predictions_twrb = trainer_twrb.predict(test_dataset_2)

In [ ]:
y_pred = predictions.predictions.argmax(axis=1)


In [ ]:
y_pred_twrb = predictions_twrb.predictions.argmax(axis=1)

In [ ]:
y_true = [x["labels"].item() for x in test_dataset]


In [ ]:
y_true_twrb = [x["labels"].item() for x in test_dataset_2]

* **Classification Reports**

Sacamos métricas del modelo *bert base uncased*

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))

[[ 602  590]
 [ 181 5781]]
              precision    recall  f1-score   support

           0       0.77      0.51      0.61      1192
           1       0.91      0.97      0.94      5962

    accuracy                           0.89      7154
   macro avg       0.84      0.74      0.77      7154
weighted avg       0.88      0.89      0.88      7154



Sacamos métricas del modelo *twitter roberta base*

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true_twrb,y_pred_twrb))
print(classification_report(y_true_twrb,y_pred_twrb))

[[ 621  571]
 [ 171 5791]]
              precision    recall  f1-score   support

           0       0.78      0.52      0.63      1192
           1       0.91      0.97      0.94      5962

    accuracy                           0.90      7154
   macro avg       0.85      0.75      0.78      7154
weighted avg       0.89      0.90      0.89      7154



Comparamos con los resultados obtenidos en el ejercicio 1, es decir usando técnicas de ingeniería de características.

* *Resultados del ejercicio 1*

                      precision    recall  f1-score   support

                0.0       0.49      0.77      0.60      1979
                1.0       0.95      0.84      0.89      9903

        accuracy                              0.83     11882
       macro avg          0.72      0.80      0.75     11882
       weighted avg       0.87      0.83      0.84     11882





* *Resultados del ejercicio 2 - modelo Bert Base Uncased*
                    
                    precision    recall  f1-score   support

                 0       0.77      0.51      0.61      1192
                 1       0.91      0.97      0.94      5962

      accuracy                               0.89      7154
      macro avg          0.84      0.74      0.77      7154
      weighted avg       0.88      0.89      0.88      7154

* *Resultados del ejercicio 2 - modelo Twitter Roberta Base Sentiment Latest*
                    
                      precision    recall  f1-score   support

                   0       0.78      0.52      0.63      1192
                   1       0.91      0.97      0.94      5962

            accuracy                           0.90      7154
          macro avg        0.85      0.75      0.78      7154
        weighted avg       0.89      0.90      0.89      7154

La conclusion es que hemos obtenido un mejor resultado usando los metodos de modelos de lenguaje que modelos de ingeniera de caracteristicas. El modelo de cardiffnlp es el que mejor metricas tiene, con una accuracy del 90%. Podemos notar sin embargo que el modelo de regresion logistica del ejercicio 1, a pesar de tener un resultado global mas bajo, tiene un mejor recall para el valor (0.77 vs 0.52).

Obviamente, ningun modelo es perfecto y cada uno de los 3 que realizamos requeriria mas tiempo para ajustar sus parametros de manera optima.